In [ ]:
import sys,tweepy
import pandas as pd
import webbrowser
from datetime import datetime, date, time, timedelta
#pip3 install tweepy
#https://docs.tweepy.org/en/latest/

##Resources Consulted:
#https://blog.f-secure.com/how-to-get-tweets-from-a-twitter-account-using-python-and-tweepy/

## Interesting Tasks

- [ ] Photo and corresponding geolocation
- [x] RT coming from which accounts. Add to account_list?
- [ ] Local accounts with GPS in WA state?

In [ ]:
#To avoid updating the Git server with secret keys and tokens, we put the sensitive information in a separate 
#notebook and add this file the .gitignore file.
%run './Authentication-Keys-and-Tokens.ipynb'

In [ ]:
def Query_Keyword_User(client,accountName,keyword,time_window=30):
    
    #client = get_twitter_client()
    
    last_search = datetime.utcnow() - timedelta(days=time_window)
    
    tweet_count=0; slide_count=0; slide_tweets = []; RT_accounts = [];
    for status in tweepy.Cursor(client.user_timeline, id=str(accountName), tweet_mode="extended").items():
        
        if(status.full_text.lower().find(str(keyword))!=-1):
            slide_tweets.append(status.full_text)
            
            if(status.full_text.find('RT')!=-1):
                tempAccountName = str(status.full_text.split()[1][1:-1])
                
                if( (tempAccountName in RT_accounts) == False):
                    RT_accounts.append(tempAccountName)
        
        if status.created_at < last_search:
            break
        
    return slide_tweets, RT_accounts


In [ ]:
if __name__ == '__main__':
    
    keywords = ["slide","debris"]
    
    
    client = get_twitter_client()
    
    slide_tweets, RT_accounts = Query_Keyword_User(client,"wsdot","slide",45)
    
    
    #Search RT_accounts for tweets including keywords and append any new tweets to slide_tweets
    for account in RT_accounts:
        temp_slide_tweets, _ = Query_Keyword_User(client,str(account),"slide",90)
        for tweet in range(len(temp_slide_tweets)):
            if( (temp_slide_tweets[tweet][:25] in slide_tweets) == False):
                slide_tweets.append(temp_slide_tweets[tweet])


In [ ]:
print(RT_accounts)
print(slide_tweets)

Previous Iterations

In [ ]:
if __name__ == '__main__':
    
    #If you want to manually enter the accounts and numbers of tweets.It would be better to define a list, I think.
    #user = input("Enter username: ")
    #num_tweets = input("Enter number of tweets: ")
    
    client = get_twitter_client()
    for status in tweepy.Cursor(client.user_timeline, id="wsdot", tweet_mode="extended").items(1):
        #print(dir(status)) #Properties/methods of the JSON returned
        print(status.full_text)
        #print(status)

In [ ]:
#Performing a Keyword Search
if __name__ == '__main__':

    client = get_twitter_client()

    account_list = ['wsdot']
    keywords = ['slide']
    time_window = 90
    
    for target in account_list:
        print("Getting data for " + target)
        item = client.get_user(target)
        print("name: " + item.name)
        print("screen_name: " + item.screen_name)
        print("description: " + item.description)
        print("statuses_count: " + str(item.statuses_count))
        print("friends_count: " + str(item.friends_count))
        print("followers_count: " + str(item.followers_count))
    
    
    last_search = datetime.utcnow() - timedelta(days=time_window)
    
    tweet_count=0; slide_count=0; slide_tweets = []; RT_accounts = [];
    for status in tweepy.Cursor(client.user_timeline, id="wsdot", tweet_mode="extended").items():
        
        tweet_count += 1
        #print(dir(status)) #Properties/methods of the JSON returned
        if(status.full_text.lower().find(keywords[0])!=-1):
            slide_count += 1
            slide_tweets.append(status.full_text)
            if(status.full_text.find('RT')!=-1):
                tempAccountName = str(status.full_text.split()[1][1:-1])
                if( (tempAccountName in RT_accounts) == False):
                    RT_accounts.append(tempAccountName)
        
        if status.created_at < last_search:
            break
        #print(status)

print(str(slide_count)+"/"+str(tweet_count)+" tweets in the previous "+str(time_window)+" days from\n"
      +str(account_list[0])+" mention \'"+keywords[0]+".'")